In [ ]:
import pandas as pd
import keras
import numpy as np
import datetime

from argparse import ArgumentParser

from lib import training as tr
from lib import database as db

In [ ]:
def get_option():
    now = datetime.datetime.now().strftime('%Y%m%d')

    a = ArgumentParser()
    a.add_argument('-k', '--kind',  choices = ['mini', 'loto6', 'loto7'], required = True)
    a.add_argument('-s', '--start', default = '19000101')
    a.add_argument('-e', '--end',   default = now)
    return a.parse_args()

In [ ]:
def strptime(d):
    return datetime.datetime.strptime(d, '%Y%m%d')

a = get_option()
kind = a.kind
loto = db.session.query(db.Loto).filter(
    db.Loto.kind == kind,
    db.Loto.date.between(strptime(a.start), strptime(a.end))
).order_by(db.Loto.times).all()

In [ ]:
def predict(kind, x):
    y = model.predict(x)
    num_classes = tr.get_num_classes(kind)

    result = {i + 1 : y[0][i] for i in range(num_classes)}
    result = sorted(result.items(), key = lambda x:-x[1])
    result = [r[0] for r in result]
    
    if kind == 'loto7':
        return sorted(result[0:7])
    elif kind == 'loto6':
        return sorted(result[0:6])
    else:
        return sorted(result[0:5])

model = keras.models.load_model('h5/' + kind + '.h5', compile = True)

num_classes = tr.get_num_classes(kind)
for i, l in enumerate(loto):
    if i < tr.x_column: continue

    x = tr.get_x_data(loto, i)
    x_test = np.array([[l.date.month, l.date.day] + x])
    x_test = x_test.astype('float32')
    x_test /= num_classes

    pred = predict(kind, x_test)
    numbers = list(map(int, l.numbers.split(',')))
    bonus = list(map(int, l.bonus.split(',')))
    
    match = list(set(pred) & set(numbers + bonus))
    print('抽選日:%s' % (l.date))
    print('　本数字　:%s ボーナス数字:%s' % (l.numbers, l.bonus))
    print('　予測数字:%s' % (','.join(map(str,pred))))
    print('　正解数　:%d個\n' % len(match))
